In [ ]:
import numpy as np
import pandas as pd

from glob import glob
from PIL import Image
import cv2

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns

from bokeh.plotting import figure
from bokeh.io import output_notebook, show, output_file
from bokeh.models import ColumnDataSource, HoverTool, Panel
from bokeh.models.widgets import Tabs

import albumentations as albu

![image](https://raw.githubusercontent.com/Lexie88rus/GlobalWheatDetection/master/wheat_image_cropped.png)

# Global Wheat Detection EDA 한글화

Computer Vision을 이용한 밀 이삭 탐지는 관련 사육업저, 농부들에게 도움이 될 것입니다!
예를 들면 :
* 작물의 생장 단계 조절 : 이삭이 열린 정도에 따라 수확일에 가까워짐
* 작물의 상태 조절 : 개수가 적거나 크기가 작을 경우 안좋은 신호
* 여러 품종의 밀에 대해 전체적인 특성과 산출량을 추정  

이 노트북을 통해 전체적인 데이터를 살펴보고, 모델을 만들고 검증하는 과정에서 유의할 내용에 대해 알아보겠습니다.

# 데이터셋에 대한 정보

숫자부터 살펴보겠습니다. 수치로부터 특별한 이미지가 나올 수도 있으니까요!

# 1. train, test 이미지의 개수:

In [ ]:
TRAIN_DIR = '../input/global-wheat-detection/train/'
TEST_DIR = '../input/global-wheat-detection/test/'
TRAIN_CSV_PATH = '../input/global-wheat-detection/train.csv'

train_fns = glob(TRAIN_DIR + '*')
test_fns = glob(TEST_DIR + '*')

train, test 이미지의 개수 계산

In [ ]:
print("Number of train images is {}".format(len(train_fns)))
print("Number of test images is {}".format(len(test_fns)))

test용 이미지가 10개밖에 없으며, 다른 test용 이미지들은 모델을 평가하기 위한 submission에서 사용됩니다.  
train을 위한 이미지로 3422개는 충분해보이지 않습니다. Data augmentation(데이터 복제)기술이 필요해보입니다.

# 2. 이미지당 박스(밀 이삭)의 개수:

이미지를 dataframe형태로 변환합니다.  
(박스가 없는 이미지는 `image_id`를 제외한 모든 열이 nan값일 것입니다.)

In [ ]:
# 박스 dataframe로드
train = pd.read_csv(TRAIN_CSV_PATH)

#train 이미지로 dataframe 생성
all_train_images = pd.DataFrame([fns.split('/')[-1][:-4] for fns in train_fns])
all_train_images.columns = ['image_id']

# 박스 dataframe과 train이미지 병합
## train이미지 위에 박스 그리기
all_train_images = all_train_images.merge(train, on='image_id', how='left')

# nan값을 0으로 대체
all_train_images['bbox'] = all_train_images.bbox.fillna('[0,0,0,0]')

# bbox 열 나누기
bbox_items = all_train_images.bbox.str.split(',', expand = True)
all_train_images['bbox_xmin'] = bbox_items[0].str.strip('[ ').astype(float)
all_train_images['bbox_ymin'] = bbox_items[1].str.strip(' ').astype(float)
all_train_images['bbox_width'] = bbox_items[2].str.strip(' ').astype(float)
all_train_images['bbox_height'] = bbox_items[3].str.strip(' ]').astype(float)

In [ ]:
print('{} images without wheat heads.'.format(len(all_train_images) - len(train)))

이미지를 살펴보겠습니다.

In [ ]:
def get_all_bboxes(df, image_id):
    image_bboxes = df[df.image_id == image_id]
    
    bboxes = []
    for _,row in image_bboxes.iterrows():
        bboxes.append((row.bbox_xmin, row.bbox_ymin, row.bbox_width, row.bbox_height))
        
    return bboxes

def plot_image_examples(df, rows = 3, cols = 3, title="Image examples"):
    fig, axs = plt.subplots(rows, cols, figsize = (10, 10))
    for row in range(rows):
        for col in range(cols):
            idx = np.random.randint(len(df), size = 1)[0]
            img_id = df.iloc[idx].image_id
            
            img = Image.open(TRAIN_DIR + img_id + '.jpg')
            axs[row, col].imshow(img)
            
            bboxes = get_all_bboxes(df, img_id)
            
            for bbox in bboxes:
                rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], linewidth = 1, edgecolor = 'r', facecolor = 'none')
                axs[row, col].add_patch(rect)
                
            axs[row, col].axis('off')
            
        plt.suptitle(title)

In [ ]:
plot_image_examples(all_train_images)

이미지의 밝기와 작물의 익은 정도가 모두 다른 것을 확인할 수 있습니다.

이미지당 박스의 개수:

In [ ]:
# train이미지당 박스의 개수 계산
all_train_images['count'] = all_train_images.apply(lambda row : 1 if np.isfinite(row.width) else 0, axis = 1)
train_images_count = all_train_images.groupby('image_id').sum().reset_index()

In [ ]:
# Bokeh를 이용한 바 차트 그리기
# https://towardsdatascience.com/interactive-histograms-with-bokeh-202b522265f3
def hist_hover(dataframe, column, colors=["#94c8d8", "#ea5e51"], bins = 30, title=''):
    hist, edges = np.histogram(dataframe[column], bins = bins)
    
    hist_df = pd.DataFrame({column:hist,
                           "left": edges[:-1],
                           "right": edges[1:]})
    hist_df["interval"] = ["%d to %d" % (left, right) for left,
                          right in zip(hist_df["left"], hist_df["right"])]
    
    src = ColumnDataSource(hist_df)
    plot = figure(plot_height = 400, plot_width = 600,
                 title = title,
                 x_axis_label = column,
                 y_axis_label = "Count")
    plot.quad(bottom = 0, top = column, left = "left",
             right = "right", source = src, fill_color = colors[0],
             line_color = "#35838d", fill_alpha = 0.7,
             hover_fill_alpha = 0.7, hover_fill_color = colors[1])
    
    hover = HoverTool(tooltips = [('Interval', '@interval'),
                                 ('Count', str("@" + column))])
    
    plot.add_tools(hover)
    
    output_notebook()
    show(plot)

In [ ]:
hist_hover(train_images_count, 'count', title = 'Number of wheat spikes per image')

대부분 이미지당 20~50개의 이삭을 갖고있습니다.

 이삭의 개수가 적은 이미지 예시를 그려보겠습니다.

In [ ]:
less_spikes_ids = train_images_count[train_images_count['count'] < 10].image_id
plot_image_examples(all_train_images[all_train_images.image_id.isin(less_spikes_ids)], title ="Example images with small number of spikes" )

이미지 예시에 이상한 점이 있습니다.
* 땅만 보이는 이미지가 있습니다.
* 많이 확대된 듯이 보이는 이미지가 있습니다.

이삭이 많은 이미지:

In [ ]:
many_spikes_ids = train_images_count[train_images_count['count'] > 100].image_id
plot_image_examples(all_train_images[all_train_images.image_id.isin(many_spikes_ids)], title = "Example images with large number of spikes")

이삭 개수가 적은 이미지보다는 나아보입니다!

# 3. 박스의 면적

In [ ]:
# 박스 면적 계산
all_train_images['bbox_area'] = all_train_images['bbox_width'] * all_train_images['bbox_height']


In [ ]:
# 박스 면적에 대한 히스토그램
hist_hover(all_train_images, 'bbox_area', title = 'Area of a single bounding box')

박스 면적의 최대값 :

In [ ]:
all_train_images.bbox_area.max()

박스 각각의 면적은 long tail 분포로 나타나고 있습니다. 면적이 넓은 이미지를 살펴보는 것이 좋을 것 같습니다.

면적이 넓은 박스 :

In [ ]:
large_boxes_ids = all_train_images[all_train_images['bbox_area'] > 200000].image_id
plot_image_examples(all_train_images[all_train_images.image_id.isin(large_boxes_ids)], title = "Example images with large bbox area")

이상하게 넓은 이 박스들은 뭘까요?? 훈련과정에서 반드시 제거해야할 것 같습니다!!

면적이 좁은 박스:

In [ ]:
min_area = all_train_images[all_train_images['bbox_area'] > 0].bbox_area.min()
print('The smallest bounding box area is {}'.format(min_area))

In [ ]:
small_boxes_ids = all_train_images[(all_train_images['bbox_area'] < 50) & (all_train_images['bbox_area'] > 0)].image_id
plot_image_examples(all_train_images[all_train_images.image_id.isin(small_boxes_ids)], title = "Example images with large bbox area")

자세히 살펴보면, 모서리와 테두리 근처에 있는 미세한 박스들을 볼 수 있습니다. 아마 경계가 먼저 그려진 뒤에, 하나의 이미지를 여러개로 잘랐기 때문에 나타난 현상같습니다.

이것들을 굳이 정리할 필요는 없어보입니다. IOU metric에 영향은 없을테니까요.
##IOU metric(Intersection over Union) - Bounding Box가 얼마나 일치하는지 나타내는 지표

# 4. 이미지당 박스의 면적

In [ ]:
# 이미지당 전체 박스의 면적 계산
area_per_image = all_train_images.groupby(by = 'image_id').sum().reset_index()

# 박스의 비율 계산
area_per_image_percentage = area_per_image.copy()
area_per_image_percentage['bbox_area'] = area_per_image_percentage['bbox_area'] / (1024 * 1024) *100

In [ ]:
hist_hover(area_per_image_percentage, 'bbox_area', title = "Percentage of image area covered by bounding boxes")

정규분포와 아주 유사한 형태로 나타나고 있습니다! 이미지 면적의 20~40%를 박스가 차지하고 있습니다.

이러한 관측은 예측 모델을 평가할 때 사용될 수 있습니다. 예측 박스의 면적 역시 정규분포를 따르겠죠.

또한 최대치가 100%을 넘어가는 것도 확인 가능합니다. 박스가 겹쳐져 있다는 의미입니다.

박스가 차지하는 면적이 좁은 이미지 :

In [ ]:
small_area_perc_ids = area_per_image_percentage[area_per_image_percentage['bbox_area'] < 7].image_id
plot_image_examples(all_train_images[all_train_images.image_id.isin(small_area_perc_ids)], title = "Example images with small percentage of area covered by bounding boxes")

박스가 차지하는 면적이 넓은 이미지 :

In [ ]:
large_area_perc_ids = area_per_image_percentage[area_per_image_percentage['bbox_area'] > 95].image_id
plot_image_examples(all_train_images[all_train_images.image_id.isin(large_area_perc_ids)], title = "Example images with large percentage of area covered by bounding boxes")

# 5. 이미지의 밝기

In [ ]:
def get_image_brightness(image):
    # grayscale(흑백)으로 전환
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # 평균 밝기 구하기
    return np.array(gray).mean()

def add_brightness(df):
    brightness = []
    for _, row in df.iterrows():
        img_id = row.image_id
        image = cv2.imread(TRAIN_DIR + img_id + '.jpg')
        brightness.append(get_image_brightness(image))
        
    brightness_df = pd.DataFrame(brightness)
    brightness_df.columns = ['brightness']
    df = pd.concat([df, brightness_df], ignore_index = True, axis = 1)
    df.columns = ['image_id', 'brightness']
    
    return df

In [ ]:
images_df = pd.DataFrame(all_train_images.image_id.unique())
images_df.columns = ['image_id']

In [ ]:
#데이터프레임의 전체적인 밝기 올리기
images_df = pd.DataFrame(all_train_images.image_id.unique())
images_df.columns = ['image_id']
brightness_df = add_brightness(images_df)

all_train_images = all_train_images.merge(brightness_df, on = 'image_id')

In [ ]:
hist_hover(all_train_images, 'brightness', title = '이미지 밝기 분포')

어두운 이미지 :

In [ ]:
dark_ids = all_train_images[all_train_images['brightness'] < 30].image_id
plot_image_examples(all_train_images[all_train_images.image_id.isin(dark_ids)], title='Darkest images')

몇몇 이미지는 사람이 보기에도 구분이 어렵네요...  

밝은 이미지:

In [ ]:
bright_ids = all_train_images[all_train_images.brightness > 130].image_id
plot_image_examples(all_train_images[all_train_images.image_id.isin(bright_ids)], title='Brightest images')

어두운 이미지랑 정반대네요. 잘보이게 해줄 필터가 필요합니다!
잘못된 바운딩 박스도 보이는 것 같네요.

#  6. 노란색, 녹색이 최대, 최소인 이미지

특정 색상이 많은 이미지를 뽑아보겠습니다. 녹색이 많다면 건강하단 의미겠죠? 노란색이 많다면 수확하기 좋은 이미지일 것이고, 갈색이 많다면 땅위에 있는 이삭일 것 같습니다!

In [ ]:
def get_percentage_of_green_pixels(image):
    #HSV로 변환
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    #녹색부분 구하기
    hsv_lower = (40, 40, 40)
    hsv_higher = (70, 255, 255)
    green_mask = cv2.inRange(hsv, hsv_lower, hsv_higher)
    
    return float(np.sum(green_mask)) / 255 / (1024 * 1024)

def get_percentage_of_yellow_pixels(image):
    #HSV로 변환
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    #녹색부분 구하기
    hsv_lower = (25, 40, 40)
    hsv_higher = (35, 255, 255)
    yellow_mask = cv2.inRange(hsv, hsv_lower, hsv_higher)
    
    return float(np.sum(yellow_mask)) / 255 / (1024 * 1024)

def add_green_pixels_percentage(df):
    green = []
    for _, row in df.iterrows():
        img_id = row.image_id
        image = cv2.imread(TRAIN_DIR + img_id + '.jpg')
        green.append(get_percentage_of_green_pixels(image))
        
    green_df = pd.DataFrame(green)
    green_df.columns = ['green_pixels']
    df = pd.concat([df, green_df], ignore_index = True, axis = 1)
    df.columns = ['image_id', 'green_pixels']
    
    return df

def add_yellow_pixels_percentage(df):
    yellow = []
    for _, row in df.iterrows():
        img_id = row.image_id
        image = cv2.imread(TRAIN_DIR + img_id + '.jpg')
        yellow.append(get_percentage_of_yellow_pixels(image))
        
    yellow_df = pd.DataFrame(yellow)
    yellow_df.columns = ['yellow_pixels']
    df = pd.concat([df, yellow_df], ignore_index = True, axis = 1)
    df.columns = ['image_id', 'yellow_pixels']
    
    return df


In [ ]:
#녹색의 픽셀의 비율 열 추가
green_pixels_df = add_green_pixels_percentage(images_df)
all_train_images = all_train_images.merge(green_pixels_df, on = 'image_id')

In [ ]:
hist_hover(all_train_images, 'green_pixels', title = '녹색 픽셀 비율 분포', colors = ['#c3ea84', '#3e7a17'])

녹색이 가장 많은 이미지의 비율은 60%입니다.

대부분의 이미지가 녹색이 전혀 없습니다! 아마 대부분 노란색으로 수확에 가깝다는 의미인 것 같습니다.

녹색이 많은 이미지:

In [ ]:
green_ids = all_train_images[all_train_images['green_pixels'] > 0.55].image_id
plot_image_examples(all_train_images[all_train_images.image_id.isin(green_ids)], title = 'The most green images')

녹색이 많은 이미지는 대부분 이제 막 피기 시작한 작은 작물들이 포함된 것 같습니다.

In [ ]:
#노란색 픽셀의 비율 열 추가
yellow_pixels_df = add_yellow_pixels_percentage(images_df)
all_train_images = all_train_images.merge(yellow_pixels_df, on = 'image_id')

In [ ]:
hist_hover(all_train_images, 'yellow_pixels', title = '노란색 픽셀 비율 분포도', colors = ['#fffedb', '#fffeab'])

노란색이 많은 이미지를 보겠습니다.

In [ ]:
yellow_ids = all_train_images[all_train_images['yellow_pixels'] > 0.55].image_id
plot_image_examples(all_train_images[all_train_images.image_id.isin(yellow_ids)], title = 'The most yellow images')

### Data Augmentation(데이터 증강)에 대한 생각(원작자)

비교적 train데이터가 적은 이번 컴피티션에서, Data augmentaion는 중요한 요소일 것 같습니다. Data augmentaion을 통해 주어진 환경 내에서 더 robust한(견고한) 모델을 만들 수 있읍니다. 어떤 augmentations/필터를 사용할 수 있을까요?:
* 원본 이미지의 방향이 제각기 다르기 때문에 *이미지를 수직 혹은 수평으로 뒤집기*
* 각각이 확대된 정도가 다르기 때문에 *크롭-리사이즈*
* 밝기를 조정하기 위한 다양한 필터. [예시 컴퍼티션](https://www.kaggle.com/c/aptos2019-blindness-detection)

주의할 점은:
* 바운딩 박스 처리가 어렵기 때문에 회전은 어려워보입니다.



augmentation 파이프라인 예시:

In [ ]:
# 예시 설치
# 바운딩박스에 영향이 없는 augmentation으로 진행
example_transforms = albu.Compose([
    albu.RandomSizedBBoxSafeCrop(512, 512, erosion_rate = 0.0, interpolation = 1, p = 1.0),
    albu.HorizontalFlip(p = 0.5),
    albu.VerticalFlip(p = 0.5),
    albu.OneOf([albu.RandomContrast(),
                albu.RandomGamma(),
                albu.RandomBrightness()], p = 1.0),
    albu.CLAHE(p = 1.0)], p = 1.0, bbox_params = albu.BboxParams(format = 'coco', label_fields = ['category_id']))

In [ ]:
def apply_transforms(transforms, df, n_transforms=3):
    idx = np.random.randint(len(df), size=1)[0]
    
    image_id = df.iloc[idx].image_id
    bboxes = []
    for _, row in df[df.image_id == image_id].iterrows():
        bboxes.append([row.bbox_xmin, row.bbox_ymin, row.bbox_width, row.bbox_height])
        
    image = Image.open(TRAIN_DIR + image_id + '.jpg')
    
    fig, axs = plt.subplots(1, n_transforms+1, figsize=(15,7))
    
    # 원본이미지
    axs[0].imshow(image)
    axs[0].set_title('original')
    for bbox in bboxes:
        rect = patches.Rectangle((bbox[0],bbox[1]),bbox[2],bbox[3],linewidth=1,edgecolor='r',facecolor='none')
        axs[0].add_patch(rect)
    
    # n번 변환 적용
    for i in range(n_transforms):
        params = {'image': np.asarray(image),
                  'bboxes': bboxes,
                  'category_id': [1 for j in range(len(bboxes))]}
        augmented_boxes = transforms(**params)
        bboxes_aug = augmented_boxes['bboxes']
        image_aug = augmented_boxes['image']

        # plot the augmented image and augmented bounding boxes
        axs[i+1].imshow(image_aug)
        axs[i+1].set_title('augmented_' + str(i+1))
        for bbox in bboxes_aug:
            rect = patches.Rectangle((bbox[0],bbox[1]),bbox[2],bbox[3],linewidth=1,edgecolor='r',facecolor='none')
            axs[i+1].add_patch(rect)
    plt.show()

In [ ]:
apply_transforms(example_transforms, all_train_images, n_transforms = 3)

In [ ]:
apply_transforms(example_transforms, all_train_images, n_transforms=3)

CLAHE가 어두운 곳을 강조하는 법을 알아두세요. 이 대회에서 꼭 필요할 것 같네요.

### 결론:

1. 이미지의 줌 레벨이 다양합니다. 모델 훈련을 위해 크롭-리사이즈 augmentation을 사용했습니다.
1. 이미지의 밝기가 다양합니다. 이를 위한 특별한 필터가 필요합니다.
1. 바운딩박스가 지저분합니다!
    모델에 훈련을 하이게 앞서 큰 박스를 정리해야 합니다.
    너무 작은 박스는 IOU 메트릭에 영향이 없으므로 둬도 될 것 같습니다.
    몇몇 이삭들은 바운딩 박스가 없습니다.
    
    